# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-08-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-10-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[:5])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['Mitch Ryder & The Detroit Wheels', 'Logged In', 'Tegan', 'F', '65', 'Levine', '205.03465', 'paid', 'Portland-South Portland, ME', 'PUT', 'NextSong', '1.54079E+12', '992', 'Jenny Take A Ride (LP Version)', '200', '1.54336E+12', '80'], ['The Spill Canvas', 'Logged In', 'Tegan', 'F', '66', 'Levine', '358.03383', 'paid', 'Portland-South Portland, ME', 'PUT', 'NextSong', '1.54079E+12', '992', 'The TIde (LP Version)', '200', '1.54336E+12', '80'], ['Mogwai', 'Logged In', 'Tegan', 'F', '67', 'Levine', '571.19302', 'paid', 'Portland-South Portland, ME', 'PUT', 'NextSong', '1.54079E+12', '992', 'Two Rights Make One Wrong', '200', '1.54336E+12', '80'], ['Spor', 'Logged In', 'Tegan', 'F', '68', 'Levine', '380.3424', 'paid', 'Portland-South Portland, ME', 'PUT', 'NextSong', '1.54079E+12', '992', 'Way Of The Samurai', '200', '1.54336E+12', '80'], ['DJ Dizzy', 'Logged In', 'Tegan', 'F', '69', 'Levine', '221.1522', 'paid', 'Portland-South Portland, ME', 'PUT', 'NextSong', '1.54079E+12', '992',

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = { 'class': 'SimpleStrategy', 'replication_factor': 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Setup a global variable
file = 'event_datafile_new.csv'

## QUERY 1: `artist_title_length` Table

### Part 1A: Create Table

In [9]:
query = "CREATE TABLE IF NOT EXISTS artist_title_length "
query = query + "(artist text, song text, length float, sessionId int, itemInSession int, \
                PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

### Part 1B: Insert Data into Table

In [10]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_title_length (artist, song, length, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (str(line[0]), str(line[9]), float(line[5]), int(line[8]), int(line[3])))

### Part 1C: Query Data from Table

In [11]:
query = """SELECT artist, song, length
           FROM artist_title_length
           WHERE sessionId = 338 AND itemInSession = 4
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(f'Artist: {row.artist} \nSong title: {row.song} \nLength: {row.length:.2f}')
    print(f'***************************')

Artist: Faithless 
Song title: Music Matters (Mark Knight Dub) 
Length: 495.31
***************************


#### Explanation:

* The table is named `artist_title_length` because the expected output should consists of these three fields.
* We chose the `Primary Key` to include `sessionId` as partition and `itemInSession` as clustering keys because these PKs should be able to give unique row to the table.
* To ensure query efficiency, we select `artist`, `song`, and `length` instead of `*`.

## QUERY 2: `artist_title_user` Table

### Part 2A: Create Table

In [12]:
query = "CREATE TABLE IF NOT EXISTS artist_title_user "
query = query + """
(artist text, song text, length float, sessionId int, itemInSession int, firstName text, lastName text, userId int,
PRIMARY KEY (userId, sessionId, itemInSession)) WITH CLUSTERING ORDER BY (sessionId ASC, itemInSession ASC);
"""

try:
    session.execute(query)
except Exception as e:
    print(e)

### Part 2B: Insert Data into Table

In [13]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_title_user \
                (artist, song, length, sessionId, itemInSession, firstName, lastName, userId)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (str(line[0]), str(line[9]), float(line[5]), int(line[8]), int(line[3]), \
                        str(line[1]), str(line[4]), int(line[10])))

### Part 2C: Query Data from Table

In [14]:
query = """
SELECT artist, song, firstName, lastName
FROM artist_title_user WHERE userId=10 and sessionId=182
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
#     print(row)
    print(f'Artist: {row.artist} \nSong: {row.song} \nFirst Name: {row.firstname} \nLast Name: {row.lastname}')
    print(f'*********************')

Artist: Down To The Bone 
Song: Keep On Keepin' On 
First Name: Sylvie 
Last Name: Cruz
*********************
Artist: Three Drives 
Song: Greece 2000 
First Name: Sylvie 
Last Name: Cruz
*********************
Artist: Sebastien Tellier 
Song: Kilometer 
First Name: Sylvie 
Last Name: Cruz
*********************
Artist: Lonnie Gordon 
Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 
First Name: Sylvie 
Last Name: Cruz
*********************


#### Explanation:

* The table is named `artist_title_user` because the expected output should consists of these four fields (ie. `users` has `firstName` and `lastName`.
* We chose the `Primary Key` to include `userId` as partition and `sessionId` and `itemInSession`. We also sort the output based on the clustering keys.
* To ensure query efficiency, we select `artist`, `song`, `firstName`, and `lastName` instead of `*`.

## QUERY 3: `song_user` Table

### Part 3A: Create Table

In [15]:
query = "CREATE TABLE IF NOT EXISTS song_user "
query = query + "(song text, firstName text, lastName text, sessionId int, PRIMARY KEY (song, sessionId))"

try:
    session.execute(query)
except Exception as e:
    print(e)

### Part 3B: Insert Data from Table

In [16]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user (song, firstName, lastName, sessionId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (str(line[9]), str(line[1]), str(line[4]), int(line[8])))

### Part 3C: Query Data from Table

In [17]:
query = """
SELECT firstName, lastName FROM song_user
WHERE song='All Hands Against His Own'
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    name = row.firstname + ' ' + row.lastname
    print(f'Name: {name}')
    print(f'***********************')

Name: Sara Johnson
***********************
Name: Jacqueline Lynch
***********************
Name: Tegan Levine
***********************


#### Explanation:

* The table is named `song_user` because the expected output should generate `user` based on `song` parameter.
* We chose the `Primary Key` to include `song` as partition and `sessionId` as clustering keys to provide unique rows.
* To ensure query efficiency, we select `firstName` and `lastName` instead of `*`. We join these two fields to improve presentation.

### Drop the tables before closing out the sessions

In [18]:
query = "drop table artist_title_length"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [19]:
query = "drop table artist_title_user"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [20]:
query = "drop table song_user"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()